## Tarea No.2 BIG DATA - Segunda Parte
### Anthony Cascante

## Dataset con resultados elecciones municipales Canton Central San Jose

In [1]:
#INSTALACION DE LIBRERIAS
import findspark
findspark.init('/opt/spark')
import pyspark

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [4]:
#from datetime import datetime
from pyspark.sql.functions import col, date_format, udf 
#from pyspark.sql.types import DateType
#from datetime import datetime

In [5]:
schema=StructType().add("DISTRITO","string").add("TOTAL","integer").add("VT","integer").add("AT", "integer").add("ATP",'integer').add("TH", "integer").add("VH", "integer").add("AH", "integer").add("AHP", "integer").add("TM", "integer").add("VM", "integer").add("AM", "integer").add("AMP", "integer")

### CARGA DE ARCHIVO DE DATOS
El archivo SANJOSE.csv posee los resultados de las elecciones municipales para la Alcaldia por el canton central de San Jose en el 2016.  Se incluyen votos recibidos por partido y abstecionismo por genero,  por cada uno de los 11 distritos del canton.




### Caracteristicas del Dataset
#### Todos los datos son por distrito

DISTRITO: Distrito

TOTAL: Total electores 

VT: Total Votos

AT: Total abstenciones

ATP: Porcentaje abstenciones

TH: Total electores hombres

VH: Total Votos hombres

AH: Total abstenciones hombres

AHP: Porcentaje abstenciones hombres

TM: Total electores mujeres

VM: Total votos mujeres

AM: Total abstenciones mujres

AMP: Porcentaje abstenciones mujeres

PAC: Votos recibidos PAC 

ASJ: Votos recibidos Alianza San Jose

TRAB: Votos recibidos P. Trabajadores

FA: Votos recibidos Frente Amplio

FUC: Votos recibidos Union Cambio

PIN: Votos recibidos Integracion Nacional

PLN: Votos recibidos Liberacion Nacional

ML: Votos recibidos Movimiento Libertario

NG: Votos recibidos Nueva Generacion

RENOV: Votos recibidos Renovacion

REPUB: Votos recibidos Republicano

SOL: Votos recibidos Solidaridad

PUSC: Votos recibidos Unidad Socialcristiano

NULOS: Votos nulos

BLANCO: Votos en blanco

In [6]:
#Carga del dataset
df3=spark.read.csv("/Users/AC/SANJOSE.csv",header=True, inferSchema=True, sep=";")

In [7]:
#Exploracion inicial del dataset - Numero de filas
df3.count()

11

In [8]:
#Exploracion inicial del dataset - Numero de columnas
len(df3.columns)

29

In [9]:
#Exploracion inicial del dataset - Verificacion del Esquema
df3.printSchema()

root
 |-- DISTRITO: string (nullable = true)
 |-- TOTAL: integer (nullable = true)
 |-- VT: integer (nullable = true)
 |-- AT: integer (nullable = true)
 |-- ATP: string (nullable = true)
 |-- TH: integer (nullable = true)
 |-- VH: integer (nullable = true)
 |-- AH: integer (nullable = true)
 |-- AHP: string (nullable = true)
 |-- TM: integer (nullable = true)
 |-- VM: integer (nullable = true)
 |-- AM: integer (nullable = true)
 |-- AMP: string (nullable = true)
 |-- PAC: integer (nullable = true)
 |-- ASJ: integer (nullable = true)
 |-- TRAB: integer (nullable = true)
 |-- FA: integer (nullable = true)
 |-- FUC: integer (nullable = true)
 |-- PIN: integer (nullable = true)
 |-- PLN: integer (nullable = true)
 |-- ML: integer (nullable = true)
 |-- NG: integer (nullable = true)
 |-- RENOV: integer (nullable = true)
 |-- REPUB: integer (nullable = true)
 |-- SOL: integer (nullable = true)
 |-- PUSC: integer (nullable = true)
 |-- NULOS: integer (nullable = true)
 |-- BLANCO: integer (n

### Algunos analisis de los datos
#### Comparacion del abstencionismo por distrito:

In [10]:
df3.select(['DISTRITO','AT']).orderBy('AT',ascending=False). show()

+--------------------+-----+
|            DISTRITO|   AT|
+--------------------+-----+
|               PAVAS|36735|
|             HATILLO|32674|
|       SAN SEBASTI�N|22564|
|            HOSPITAL|14223|
|SAN FRANCISCO DE ...|13298|
|              ZAPOTE|13157|
|            CATEDRAL|11166|
|               URUCA|11156|
|              MERCED| 7091|
|        MATA REDONDA| 5070|
|              CARMEN| 3401|
+--------------------+-----+



#### Comparacion del abstencionismo por distrito por genero:

In [11]:
df3.select(['DISTRITO','AHP','AMP']). show()

+--------------------+----+----+
|            DISTRITO| AHP| AMP|
+--------------------+----+----+
|              CARMEN|70,3|73,7|
|              MERCED|73,4|71,0|
|            HOSPITAL|76,3|70,9|
|            CATEDRAL|73,4|73,5|
|              ZAPOTE|73,7|72,7|
|SAN FRANCISCO DE ...|72,6|71,9|
|               URUCA|74,8|73,3|
|        MATA REDONDA|69,1|67,7|
|               PAVAS|75,2|72,6|
|             HATILLO|75,5|73,5|
|       SAN SEBASTI�N|75,9|73,9|
+--------------------+----+----+



### Comparacion entre candidato ganador, PLN, PAC, PUSC, ABSTENCIONISMO

In [12]:
df3.select(['DISTRITO','PAC','PLN', 'ASJ','PUSC','AT']).show()

+--------------------+---+----+----+----+-----+
|            DISTRITO|PAC| PLN| ASJ|PUSC|   AT|
+--------------------+---+----+----+----+-----+
|              CARMEN|116| 222| 340| 187| 3401|
|              MERCED|123| 459|1028| 221| 7091|
|            HOSPITAL|272| 856|2459| 424|14223|
|            CATEDRAL|353| 672|1313| 591|11166|
|              ZAPOTE|327| 811|1578| 495|13157|
|SAN FRANCISCO DE ...|329| 764|1612| 627|13298|
|               URUCA|288| 784|1199| 410|11156|
|        MATA REDONDA|149| 450| 737| 356| 5070|
|               PAVAS|551|1969|4761|1499|36735|
|             HATILLO|347|1555|5471| 802|32674|
|       SAN SEBASTI�N|336| 994|3427| 710|22564|
+--------------------+---+----+----+----+-----+



#### Comparacion entre total de votos de los cuatro partidos con mas votacion (total4) Vrs Abstencionismo (AT)

In [13]:
columnsToSum = (col("PAC"), col("PLN"), col("ASj"), col("PUSC"))
newdf = df3.withColumn('total4', sum(columnsToSum))
columnsToSum

(Column<b'PAC'>, Column<b'PLN'>, Column<b'ASj'>, Column<b'PUSC'>)

In [14]:
newdf.select(['DISTRITO','AT','total4']).show()

+--------------------+-----+------+
|            DISTRITO|   AT|total4|
+--------------------+-----+------+
|              CARMEN| 3401|   865|
|              MERCED| 7091|  1831|
|            HOSPITAL|14223|  4011|
|            CATEDRAL|11166|  2929|
|              ZAPOTE|13157|  3211|
|SAN FRANCISCO DE ...|13298|  3332|
|               URUCA|11156|  2681|
|        MATA REDONDA| 5070|  1692|
|               PAVAS|36735|  8780|
|             HATILLO|32674|  8175|
|       SAN SEBASTI�N|22564|  5467|
+--------------------+-----+------+



### Votos total candidato ganador Vrs Abstencionismo

In [15]:
totalAT = df3.groupBy().agg(F.sum("AT")).collect()
totalASJ = df3.groupBy().agg(F.sum("ASJ")).collect()
total = df3.groupBy().agg(F.sum("TOTAL")).collect()

In [16]:
print("TOTAL ABSTENCIONES:",totalAT)
print("TOTAL VOTOS GANADOR:",totalASJ)
print("TOTAL ELECTORES:",total)

TOTAL ABSTENCIONES: [Row(sum(AT)=170535)]
TOTAL VOTOS GANADOR: [Row(sum(ASJ)=23925)]
TOTAL ELECTORES: [Row(sum(TOTAL)=231682)]


In [17]:
columnsNames = (col("PAC"), col("PLN"))